# Module 1: Introduction to Exploratory Analysis 

In this module, we explore our data. Exploring data is a critical piece of data science that can be as extensive as you like. Here, we explore questions that we thought were interesting, but you certainly can and we encourage you to take exploration in the direction that you like. 

## Step 1: Import and Merge Data

At the beginning of our notebooks, we will always import all the libraries we will use so as to keep things organized. 

In [8]:
import pandas as pd
import numpy as np
from ggplot import *
import matplotlib.pyplot as plt
from datetime import datetime
import dateutil.parser
pd.options.display.mpl_style = 'default'

# the matplotlib inline command is important, it tells jupyter notebook to show the output of the cell for charts
%matplotlib inline

/Users/Sara/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: 
mpl_style had been deprecated and will be removed in a future version.
Use `matplotlib.pyplot.style.use` instead.

  exec(code_obj, self.user_global_ns, self.user_ns)


You are using Jupyter notebook to run your code. it is an incredibly versatile tool and we can tailor the configurations to suit our preferences. For example, in the cell below we tell Jupyter to output the results of all commands in a cell (the default is only to output the results of the last command). For more interesting and incredibly useful Jupyter tricks take a look at [this](https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/) blog post. Using Jupyter shortcuts will help you work faster. Some valuable shortcuts have been put together in [this](http://www.math.umassd.edu/~ahausknecht/aohWebsiteSpring2017/examples/pythonExamples/downloads/docs/JuypterKeyboardShortcutsV1Sp2016.pdf) great pdf.

In [9]:
# the command below means that the output of multiple commands in a cell will be output at once.

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

We use a command from the Pandas library (a Python library that makes many aspects of data analysis easier) to tailor how the data is presented in the command below. There are 76 columns in our final Kiva dataset. The default settings hide some of the columns. We make all columns visible by setting the display to 80.

In [10]:
# the command below tells jupyter to display up to 80 columns, this keeps everything visible
pd.set_option('display.max_columns', 80)
pd.set_option('expand_frame_repr', True)

We will use three main data sets: <b>loans_full.csv, loans_details.csv, loans_partner_details.csv</b>. These are all datasets KIVA makes publically available through its API. For further details on how to pull your own data, see the Delta Analytics GitHub. For now, we will use this dataset throughout the entire course so you can match your output to ours. 

Let's start by talking a little about each data set:

[loans_full](http://build.kiva.org/api#GET*|loans|search): contains all successful and expired loans made by KIVA in Kenya. KIVA does not make public loans that have defaulted in order to protect the privacy of those users.

[loans_details](http://build.kiva.org/api#GET*|loans|:ids): contains many of the coumns as loans full. However, it also contains a comprehensive description of the lender.

[loans_partner_details](http://build.kiva.org/api#GET*|partners|:ids): detailed performance metrics associated with each partner. Includes name of partner, rating of partner, amount lent to partner so far. Note that a partner may not necessarily only be operating in Kenya, so the total loan amounts and number of loans may not match with the aggregate amounts in loan_details or loan_ful.


When we import all out data sets we will also import a data dictionary with definitions of each field. We talk about the data dictionary more below.

We set the data path below. 

<b>NOTE</b>: You should update this path to have the name of your computer!

In [11]:
data_path = 'C://Users/JANE_DOE/Desktop/intro_course_data_science_for_good/data'

We import all three data sets in the cell below.

In [12]:
loans_full=pd.read_csv(data_path+'/loans_full.csv', low_memory=False)
loans_details=pd.read_csv(data_path+'/loans_details.csv', low_memory=False)
loan_partner_details=pd.read_csv(data_path+'/loans_partner_details.csv', low_memory=False)

We will also import our data dictionary which contains the definition for each of the fields in the final data set we will use. Reference the data dictionary for important context about the data. It may be easier to open it as a csv file in your desktop.

In [13]:
kiva_data_dictionary=pd.read_csv(data_path+'/kiva_data_dictionary.csv')

In [14]:
kiva_data_dictionary.head(100)

,original_data_set,field_name,field_name_final_dataset,description,additional_notes
0,loans_full,id,id,Unique identifier for a Kiva loan,Int
1,loans_full,name,name,The name of the borrower or group receiving th...,String
2,loans_full,gender,gender,The gender of the primary borrower OR majority...,possible values are male or female
3,loans_full,description,description,The description of the loan profile in English.,String
4,loans_full,status,status,All possible loan statuses,The public dataset does not reveal defaulted l...
5,loans_full,fundedAmount,fundedAmount,​The amount of the loan that has been purchase...,​The dollar amount in US
6,loans_full,basketAmount,basketAmount,The amount of the unfunded loan that is sittin...,​The dollar amount in US
7,loans_full,image,image,The picture for this loan profile,Image
8,loans_full,activity,activity,The activity is a structured categorization of...,A property of loan which is more descriptive t...
9,loans_full,sector,sector,The sector is a more general classification of...,"A sector is a broad category for a loan, e.g. ..."


## Merging Data

In the cell below we perform a very important piece of code. We merge our datasets together in order to have a single final dataset. You will often have to merge/join datasets because many databases are relational databases. This means data is stored in tables that can be accessed and/or reassembled in many different ways (you can learn more [here](http://searchsqlserver.techtarget.com/definition/relational-database)). This has important benefits:
- allowing the flexible addition of new data without exsisting tables needing to be altered
- limiting the size of the data we store

Instead of having one single huge table in our database, we have many different tables that we can piece together as needed. This act of piecing together datasets is called a merge.


So when does it make sense to merge data?

In order to merge data, you need to have a columns in both datasets that have shared values. The columns you use to perform the join are called <b>"join keys."</b>

For example, the join key in both loans_full and loans_details is the loan ID. This is the unique identifier for a loan and is shared between both datasets. In both datasets this is a unique key, which means we only expect a single row for every row (no duplicates).

The join key between loans_full and loans_partner_details is the partner_id. This is unique in loans_partner_details (every row is a unique partner_id), but is not unique in loans_full since many rows (loans) belong to the same partner. We can show that is not unique below by doing a value_count of the number of loans that belong to each partner_id in loans_full. We show the top 10 loan partners by size, you can see that the top loan partner '133' has been involved in 28,560 loans. When we join loans_full to loans_partner_details we can associate a name with each number identifier, and it will be a lot more interesting.



In [15]:
loans_full['partner_id'].value_counts().nlargest(10)

133.0    28560
164.0    21552
138.0    18101
156.0    18081
202.0    10356
142.0     5922
388.0     3210
203.0     2987
386.0     2388
6.0       1102
Name: partner_id, dtype: int64

There are a few different types of merges, and the type of merge you choose depends upon what data you want to keep in your final data set.

There are two fantastic blog posts which do a great job explaining the different types of merges [here](http://www.datacarpentry.org/python-ecology-lesson/04-merging-data/) and [here](https://blog.codinghorror.com/a-visual-explanation-of-sql-joins/). We will reference the language in both posts below to add intuition.


**inner join** 

We will be using an inner join to join loans_full and loans_details because we only want the rows that match in both the original dataframes.

Inner joins yield a DataFrame that contains only rows where the value being joins exists in BOTH tables. The venn diagram below shows what is kept from both data frames.

![Visual of an inner join](module_1_images/inner_join.png)

Below we test it in python using our two datasets, loans_full and loan_difference.

loans_full and loan_difference contain many columns that are duplicates (exist in both). There is no need to merge all the columns so we start out by figuring out what additional columns ARE in loans_details that ARE NOT in loans_full. We can see below that only borrower_count is additional, so we only merge these two columns.

In [16]:
additional_columns = loans_full.columns.difference(loans_details.columns).tolist() + ['id']
print(additional_columns)

['borrower_count', 'id']


Finally, we are ready to merge. We do not need to specify inner join since it is the default option. However, we do need to specify what are the join keys in each data set. In this case the naming is the same in both datasets, so we specify 'id' as the join key for the loans_details data set and 'id' for the loans_full data set.

In [17]:
df = loans_details.merge(loans_full[['borrower_count', 'id']], left_on='id', right_on='id')

In [18]:
len(df)

127958

You can take a look at the output below and confirm that borrow_count is now there. The new dataframe only contains loans that are in both loans_full and loans_details.

In [19]:
# The sample() function takes a random sample from the specified data frame. 
df.sample(3)

,Unnamed: 0,activity,basket_amount,bonus_credit_eligibility,borrowers,currency_exchange_loss_amount,description.languages,description.texts.en,description.texts.es,description.texts.fr,description.texts.ru,funded_amount,funded_date,id,image.id,image.template_id,journal_totals.bulkEntries,journal_totals.entries,lender_count,loan_amount,location.country,location.country_code,location.geo.level,location.geo.pairs,location.geo.type,location.town,name,partner_id,payments,planned_expiration_date,posted_date,sector,status,tags,terms.disbursal_amount,terms.disbursal_currency,terms.disbursal_date,terms.loan_amount,terms.local_payments,terms.loss_liability.currency_exchange,terms.loss_liability.currency_exchange_coverage_rate,terms.loss_liability.nonpayment,terms.repayment_interval,terms.repayment_term,terms.scheduled_payments,themes,translator.byline,translator.image,use,video.id,video.thumbnailImageId,video.title,video.youtubeId,borrower_count
80715,80663,Food Stall,NaN,False,"[{'first_name': 'Elias', 'last_name': '', 'gen...",NaN,['en'],Elias lives in the Mariakani area of Coast Pro...,NaN,NaN,NaN,250,2013-06-06T23:22:17Z,564537,1358807,1,0,0,8,250,Kenya,KE,town,1 38,point,Likoni,Elias,203.0,[],2013-07-03T09:30:03Z,2013-06-03T09:30:03Z,Food,funded,[],21100.0,KES,2013-05-15T07:00:00Z,250,[],shared,0.1,lender,NaN,7,[],NaN,NaN,NaN,"to buy food items, such as wheat flour, cookin...",NaN,NaN,NaN,NaN,1
86084,86028,Water Distribution,NaN,True,"[{'first_name': 'Rukiya', 'last_name': '', 'ge...",NaN,['en'],Rukiya has been married for several years. She...,NaN,NaN,NaN,575,2013-02-16T01:31:32Z,527227,1293712,1,0,0,15,575,Kenya,KE,town,1 38,point,Kisauni,Rukiya,164.0,[],2013-03-17T05:00:03Z,2013-02-15T05:00:03Z,Services,funded,[],50000.0,KES,2013-01-17T08:00:00Z,575,[],shared,0.1,lender,NaN,14,[],NaN,Kate McKinna,NaN,to buy a large water tank to store enough wate...,NaN,NaN,NaN,NaN,1
9087,9087,Farming,NaN,False,"[{'first_name': 'Gladys', 'last_name': '', 'ge...",NaN,['en'],"Pictured above is Gladys, who, as a group lead...",NaN,NaN,NaN,550,2017-01-29T20:16:54Z,1214908,2368066,1,0,0,21,550,Kenya,KE,town,1 38,point,Matete,Gladys' Group,202.0,[],2017-02-03T15:10:05Z,2017-01-04T15:10:05Z,Agriculture,funded,"[{'name': 'user_favorite'}, {'name': '#Sustain...",55140.0,KES,2017-02-01T08:00:00Z,550,[],shared,0.1,lender,NaN,11,[],"['Green', 'Rural Exclusion', 'Earth Day Campai...",Lynn Cerra,860417.0,to purchase hybrid seeds and fertilizer to imp...,NaN,NaN,NaN,NaN,8


**left join**

Now we perform a different type of merge in order to incorporate the loan_partner_details dataset. This merge is called a left join. The description below is directly from [this](http://www.datacarpentry.org/python-ecology-lesson/04-merging-data/) blog post.

Like an inner join, a **left join** uses join keys to combine two DataFrames. Unlike an inner join, a left join will return all of the rows from the left DataFrame, even those rows whose join key(s) do not have values in the right DataFrame. Rows in the left DataFrame that are missing values for the join key(s) in the right DataFrame will simply have null (i.e., NaN or None) values for those columns in the resulting joined DataFrame.

Note: a left join will still discard rows from the right DataFrame that do not have values for the join key(s) in the left DataFrame.

The venn diagram below shows what is kept from both data frames.

![Visual of an left join](module_1_images/left_join.png)

What this means is that we will keep our entire df dataframe, and we will only pull in data from loan_partner_details of the partner_id for partner_id listed in df. We will discard everything else. The code below goes through how this is done.

In the cell below, we rename all the columns in loans_partner_details with the prefix partner in order to avoid confusion once we merge. Then we list all columns that are in loans_partner_details but not in df. These are the new columns we want to merge. Finally, we do a left join.

A left join is performed in pandas by calling the same merge function used for inner join, but using the how='left' argument.

In [20]:
new_names = [(i,'partner_'+i) for i in loan_partner_details.iloc[:, 1:].columns.values]
loan_partner_details.rename(columns = dict(new_names), inplace=True)

In [21]:
additional_columns = loan_partner_details.columns.difference(df.columns).tolist() + ['partner_id']
print(additional_columns)

['partner_average_loan_size_percent_per_capita_income', 'partner_charges_fees_and_interest', 'partner_countries', 'partner_currency_exchange_loss_rate', 'partner_default_rate', 'partner_default_rate_note', 'partner_delinquency_rate', 'partner_delinquency_rate_note', 'partner_image.id', 'partner_image.template_id', 'partner_loans_at_risk_rate', 'partner_loans_posted', 'partner_name', 'partner_portfolio_yield', 'partner_portfolio_yield_note', 'partner_profitability', 'partner_rating', 'partner_social_performance_strengths', 'partner_start_date', 'partner_status', 'partner_total_amount_raised', 'partner_url', 'partner_id']


In [22]:
df = df.merge(loan_partner_details[['partner_average_loan_size_percent_per_capita_income', 
                                    'partner_charges_fees_and_interest', 'partner_countries', 
                                    'partner_currency_exchange_loss_rate', 'partner_default_rate', 
                                    'partner_default_rate_note', 'partner_delinquency_rate', 'partner_delinquency_rate_note', 
                                    'partner_image.id', 'partner_image.template_id', 'partner_loans_at_risk_rate', 'partner_loans_posted', 
                                    'partner_name', 'partner_portfolio_yield', 'partner_portfolio_yield_note', 'partner_profitability', 
                                    'partner_rating', 'partner_social_performance_strengths', 'partner_start_date', 'partner_status', 
                                    'partner_total_amount_raised', 'partner_url', 'partner_id']], how='left', left_on='partner_id', right_on='partner_id') 


In [23]:
len(df)

127958

We have successully merged our dataframe and we are ready to get started with our exploratory analysis. For completeness, below is a short description of the other join types you are likely to come across as you work with data.

**full join**

![Visual of an full join](module_1_images/full_join.png)

A full join returns all rows from both datasets even if there is no match. All records will be kept so this join is used when we do not want to lose any data but want to see what rows match.
Invoked by passing how='outer' as an argument. 

**right join**

A right join is the same concept as the left join except it keeps all rows in the right data set rather than the left. Invoked by passing how='right' as an argument. 


### Export final dataset

We don't want to have to join all our datasets everytime we get started, so we will save our work to our local data folder using the command below.

In [24]:
df.to_csv(data_path+'/df.csv')